In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 200 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "tayyibsupercool/Phi_3.5_mini-resource_allocation-energy_efficiecy_instruct_10k",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.1.5: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/120M [00:00<?, ?B/s]

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(intructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("tayyibsupercool/resource_allocation_telecom_energy_efficiency_instruct", split = "validation[0:10000]")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from unsloth import FastLanguageModel

# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# instruction to generate two ouputs
instruction = "Take a deep breath and work on this problem step-by-step. You are a mathematical tool to predict some model. Your job is to predict B for given A. Give two predictions of B for each A. The following is the dataset that you can use for the predictions."

inputs = tokenizer(
[
    alpaca_prompt.format(
        instruction, # instruction
        dataset[0]['input'], # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nTake a deep breath and work on this problem step-by-step. You are a mathematical tool to predict some model. Your job is to predict B for given A. Give two predictions of B for each A. The following is the dataset that you can use for the predictions.\n\n### Input:\nIf A is -56, -352, -172, -32,\n\n### Response:\n0, 49.<|endoftext|>']

In [ ]:
from tqdm import tqdm  # Import tqdm for the progress bar
import json
import torch

# List to store the results
results = []

# Batch size for inference
batch_size = 8  # Adjust as needed based on your GPU memory

# Process the dataset in batches
for i in tqdm(range(0, len(dataset), batch_size), desc="Generating Responses", unit="batch"):
    # Convert the batch to a list of dictionaries
    batch_examples = dataset[i:i + batch_size]  # This returns a dictionary of lists
    batch_examples = [dict(zip(batch_examples, t)) for t in zip(*batch_examples.values())]  # Convert to list of dictionaries

    # Format the prompts for the batch
    input_texts = [
        alpaca_prompt.format(example['instruction'], example['input'], "")
        for example in batch_examples
    ]

    # Tokenize the input and move to GPU
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate multiple responses for each input
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=64,
            use_cache=True,
            num_return_sequences=3,  # Generate three responses per input
            do_sample=True,         # Enable sampling for diverse outputs
            top_k=50,               # Use top-k sampling to add randomness
            temperature=0.7         # Control randomness in generation
        )

    # Decode the batch outputs
    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # print(generated_texts)
    # Group generated texts by input (since num_return_sequences > 1)
    grouped_texts = [
        generated_texts[j:j + 3] for j in range(0, len(generated_texts), 3)
    ]

    # Extract responses and append to results
    for example, generated_pair in zip(batch_examples, grouped_texts):
        response_1, response_2, response_3 = generated_pair

        # Process responses for consistent formatting
        response_1 = response_1.split("### Response:\n")[1].strip() if "### Response:\n" in response_1 else response_1.strip()
        response_2 = response_2.split("### Response:\n")[1].strip() if "### Response:\n" in response_2 else response_2.strip()
        response_3 = response_3.split("### Response:\n")[1].strip() if "### Response:\n" in response_3 else response_3.strip()

        # Add the original input, original response, and two predicted responses to the results
        results.append({
            "sample_index": example["sample_index"],  # Include the sample index
            "instruction": example["instruction"],
            "input": example["input"],
            "original_response": example["output"],
            "predicted_response_1": response_1,
            "predicted_response_2": response_2,
            "predicted_response_3": response_3
        })

# Save the results to a JSON file
output_filename = "generated_responses_ee_with_three_responses.json"
with open(output_filename, "w") as outfile:
    json.dump(results, outfile, indent=4)

print(f"Results saved to {output_filename}")
